In [1]:
import numpy as np
import pandas as pd
import gc
import os, shutil
from tqdm import *
# import ipyvolume as ipv
import open3d as o3d

from multiprocessing import Pool, cpu_count
from struct import *
import time
from lidar_point_pb2 import *
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import argparse
import pickle
%matplotlib notebook


In [2]:
BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/iisc/2020-02-10"

STATIC_DATA_FOLDER = "2020-02-10-15-32-49-p15-auto-5loops_static"
DYNAMIC_2_DATA_FOLDER = "2020-02-10-15-57-38-p15-auto-5loops_dy_2"
DYNAMIC_1_DATA_FOLDER = "2020-02-10-16-12-42-p15-auto-5loops_dy_1"

STATIC_DATA_PATH = os.path.join(BASE_PATH, STATIC_DATA_FOLDER, "debug")
DYNAMIC_1_DATA_PATH = os.path.join(BASE_PATH, DYNAMIC_1_DATA_FOLDER, "debug")
DYNAMIC_2_DATA_PATH = os.path.join(BASE_PATH, DYNAMIC_2_DATA_FOLDER, "debug")

DATA_PATH = STATIC_DATA_PATH

In [3]:
PCD_FOLDER = 'complete_extracted'
PCD_PATH = os.path.join(DATA_PATH, PCD_FOLDER)
if not os.path.exists(PCD_PATH):
    os.makedirs(PCD_PATH)
else:
    shutil.rmtree(PCD_PATH)
    os.makedirs(PCD_PATH)

In [4]:
def getint(name):
    str_list = list(filter(str.isdigit, name))
    if len(str_list) == 1:
        return int(str_list[0])
    elif len(str_list) == 0:
        return 0
    else:
        print("String contains more than 1 integer")
        assert False

In [5]:
def read_packet(f):
    hdr = f.read(4)
    if hdr:
        #size, = struct.unpack("I", hdr)
        size, = unpack("I", hdr)
        # print(size) #Size is 488556
        #lp = ati.schema.LidarPoint()
        lp = LidarPoint()
        lp.ParseFromString(f.read(size))
        return lp

def read_frames(file_name):
    #file_name = "lidar-0"
    with open(file_name, "rb") as proto_file:
        while True:
            lp = read_packet(proto_file)

            if lp is None:
                break

            va = np.frombuffer(lp.vertical_angles, dtype=np.float64)
            ha = np.frombuffer(lp.horizontal_angles, dtype=np.float64)
            d = np.frombuffer(lp.distances, dtype=np.float64)
            i = np.frombuffer(lp.intensities, dtype=np.float64)

            beams = va.shape[0]
            num_points = d.shape[0]

            va = np.tile(va, num_points // beams).reshape(-1)
            ha = np.repeat(ha, beams).reshape(-1)
            d_v_cos = d * np.cos(va)

            frame = np.vstack([
                d_v_cos * np.sin(ha), # X
                d_v_cos * np.cos(ha), # Y
                d * np.sin(va),  # Z
                d, va, ha, i
            ]).T
            
            yield (lp.time, frame)

In [6]:
def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.show()

In [7]:
def parallel_thread(idx):
    temp_df = df_frames[idx_list[idx]: idx_list[idx+1]]
    single_pcd_arr = np.array([np.array([row['x'], row['y'], row['z']]) for idx, row in temp_df.iterrows()])
    
    single_pcd = o3d.geometry.PointCloud()
    single_pcd.points = o3d.utility.Vector3dVector(single_pcd_arr)
    
    single_pcd_fname = str(pcd_name_idx + idx + 1) + ".pcd"
    single_pcd_path = os.path.join(DATA_PATH, 'complete_extracted', single_pcd_fname)
    o3d.io.write_point_cloud(single_pcd_path, single_pcd, write_ascii=True)

In [17]:
old_theta = 7
pcd_name_idx = 0

pb_files = sorted([file for file in os.listdir(DATA_PATH) if file[:5] == 'lidar' and file.endswith(".pb")], key=getint)
# for saby in range(1):
for pb_file in pb_files:
    print('Reading frames from {}'.format(pb_file))
    frm_list = []
    tym_list = []
    for tym, frm in tqdm_notebook(read_frames(os.path.join(DATA_PATH, pb_file))):
        tym_list.append(np.ones(frm.shape[0])*tym)
        frm_list.append(frm)
    gc.collect()
    frm_arr = np.concatenate(frm_list)
    tym_arr = np.concatenate(tym_list)
    del frm_list, tym_list
    gc.collect()
    
    print('Putting to dataframe (redundant)')
    df_frames = pd.DataFrame(frm_arr).rename(columns={0:'x', 1:'y', 2:'z', 3:'r', 4:'phi', 5:'tht', 6:'i'})
    df_frames['t'] = tym_arr
    del frm_arr, tym_arr
    gc.collect()
    
    print("Cleaning up undetected points")
    print(df_frames[~(df_frames['r'] == 0)].shape[0] / df_frames.shape[0])
    df_frames = df_frames[~(df_frames['r'] == 0)]
    
    print("Finding single revolutions")
    idx_list = []
    for idx, theta in tqdm_notebook(enumerate(df_frames['tht']), total=df_frames.shape[0]):
        if old_theta > theta:
            idx_list.append(idx)
        old_theta = theta
    gc.collect()
    
    print("Saving to pcds")
    parallel_args = range(len(idx_list) - 1)
#     process_pool = Pool(cpu_count()-1)
    process_pool = Pool(12)
    # process_pool = Pool(1)
    score_list = [each for each in tqdm_notebook(process_pool.imap_unordered(parallel_thread, parallel_args),
                                                 total = len(parallel_args))]
    process_pool.terminate()
    gc.collect()
    
    # pcd count in this pb file
    pcd_name_idx = pcd_name_idx + len(idx_list) - 1
    gc.collect()
    
    print('{} converted to {} pcd files'.format(pb_file, len(idx_list)-1))
    print("****************************************************************")


Reading frames from lidar-1.pb


/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
len(idx_list)-1

In [ ]:
%matplotlib notebook

## Extract complete revolution